In [212]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from helpers import get_abspath

# generate individual algorithm plots
resdir = 'results/NN'
BP = pd.read_csv(get_abspath('BP/results.csv', resdir))

In [214]:
BP

,iteration,MSE_train,MSE_validation,MSE_test,acc_train,acc_validation,acc_test,seconds_elapsed
0,0,1.712192e-01,0.157870,0.161995,0.649378,0.674033,0.665806,1.107001
1,10,1.973956e-02,0.041475,0.036053,0.952974,0.908840,0.923871,17.466366
2,20,5.312103e-03,0.042998,0.040711,0.986860,0.906077,0.907097,31.849095
3,30,2.819273e-03,0.044927,0.042412,0.992393,0.897790,0.907097,47.973685
4,40,1.590289e-03,0.045953,0.043713,0.995159,0.895028,0.899355,64.380411
5,50,7.995323e-04,0.045671,0.042755,0.997925,0.895028,0.901935,79.921691
6,60,5.216435e-04,0.044815,0.041090,1.000000,0.903315,0.905806,94.463013
7,70,2.608453e-04,0.044659,0.040706,1.000000,0.895028,0.908387,109.395725
8,80,1.657811e-04,0.042581,0.041274,1.000000,0.903315,0.908387,124.530241
9,90,7.314424e-05,0.042658,0.042002,1.000000,0.903315,0.904516,139.395598


In [45]:
# get columns
iterations = df['iteration']
MSE_train = df['MSE_train']
MSE_test = df['MSE_test']
MSE_valid =df['MSE_validation']

# create error curve
plt.figure(0)
plt.plot(iterations, MSE_train, color='b', label='Training')
plt.plot(iterations, MSE_test, color='r', label='Validation')
plt.plot(iterations, MSE_valid, color='g', label='Test')
plt.legend(loc='best')
plt.grid(linestyle='dotted')
plt.xlabel('Iterations')
plt.ylabel('Mean squared error')
# plt.ylim((0, 1))

# save learning curve plot as PNG
plotdir = 'plots\\NN'
plot_tgt = '{}\\{}'.format(plotdir, oaName)
plotpath = get_abspath('{}_error.png'.format(oaName), plot_tgt)
plt.savefig(plotpath, bbox_inches='tight')
plt.close()